In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import time
import string
import math

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize as wt
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertConfig, TFBertModel
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.metrics import RootMeanSquaredError
import xgboost as xgb
import tensorflow as tf

stemmer = PorterStemmer()
from IPython.core.display import display
from scipy import stats
from scipy.spatial import distance
import matplotlib.pyplot as plt
import missingno as msno
import string

import spacy

nlp = spacy.load('en_core_web_sm')

hrule = lambda x: "=" * x
stop_words = stopwords.words('english')

# Introduzione

Il seguente notebook è stato creato per la partecipazione ad una competizione, ovvero CommonLit Readability, sulla piattaforma Kaggle. L'obiettivo della competizione consiste nel creare un modello per valutare la complessità di estratti di testo per l'uso in classe di grado 3-12 (sistema scolastico americano).

Viene fornito un trainset con estratti di testo e target (che indica la facilità di lettura) e un testset con soli estratti di testo.

Per la valutazione del modello nella competizione viene usato la radice dell'errore quadratico medio dei valori target predetti sul testset.

# Caricamento dei dati

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

# Analisi dei dati

Informazioni di base sul dataset

In [ ]:
print(hrule(50))
print("Dataset shape:")
print(train.shape)
print(hrule(50))
train.info()
print(hrule(50))

In [ ]:
msno.bar(train)
plt.show()

Percentuale di valori nulli:

In [ ]:
pd.DataFrame(train.isnull().sum() / train.shape[0]).T

In [ ]:
subset = train[train['license'].isna() & (train['url_legal'].isna() == False) |
               (train['license'].isna() == False) & train['url_legal'].isna()]
print("Numero di occorrenze in cui url_legal è mancante e license è dato o viceversa: " + str(len(subset)))

In [ ]:
valoriMancanti = train.isnull().sum() / len(train)
valoriMancanti = valoriMancanti[valoriMancanti > 0]
print("Righe in cui mancano url_legal e license: " + str(int(valoriMancanti[0] * 100)) + "%")

Alcuni dettagli statistici del dataset

In [ ]:
train.describe()

Note
* I valori mancanti sono solo nelle colonne url_legal e license, entrambe le colonne hanno per il 70% valori nulli.
* License si riferisce a url_legal, dove manca un dato, mancherà anche l'altro.

In [ ]:
test

In [ ]:
msno.bar(test)
plt.show()

In [ ]:
subset = test[test['license'].isna() & (test['url_legal'].isna() == False) |
              (test['license'].isna() == False) & test['url_legal'].isna()]
print("Numero di occorrenze in cui url_legal è mancante e license è dato o viceversa: " + str(len(subset)))

In [ ]:
valoriMancanti = test.isnull().sum() / len(test)
valoriMancanti = valoriMancanti[valoriMancanti > 0]
print("Righe in cui mancano url_legal e license: " + str(int(valoriMancanti[0] * 100)) + "%")

Note
* Il test set ha solo 7 righe
* Non contiene standard error.

## Analisi del valore target

Il target rappresenta la facilità di lettura di un testo. Un target alto indica un testo di facile lettura destinato a
classi di grado più basso.

In [ ]:
train.query('target == 0')

Possiamo notare che in questa particolare riga il target e lo standard_error è 0, questo perché il testo di riferimento
è usato come comparatore, per cui tutte le altre righe hanno ricevuto diverse classificazioni in base alla facilità di
lettura della riga con id *436ce79fe*. In particolare, le righe sono state valutate in base al modello
Bradley-Terry [1]. Se un testo è più facile da leggere rispetto al testo con id *436ce79fe*, allora questo riceverà un
 valore positivo, altrimenti sarà negativo.

In [ ]:
we = stats.probplot(train.target, plot=plt)
plt.show()

In [ ]:
sns.histplot(train.target, kde=True, stat="density").set_title('Target distribution')
plt.show()

Ordinazione crescente dei dati in base al target

In [ ]:
display(train.sort_values(by=['target']).head())

Ordinazione decrescente dei dati in base al target

In [ ]:
display(train.sort_values(by=['target'], ascending=False).head())

Ordinando il dataset in base ai valori del target vediamo che questo assume dei valori compresi tra -3.676268 e 1.711390. E' inoltre possibile notare che i campi url_legal e license sono presenti soprattutto nei casi dove il valore del target è più alto.

### Riferimenti
[1] https://www.kaggle.com/gunesevitan/commonlit-readability-prize-eda

## Analisi degli estratti

Analisi del numero di parole negli estratti di testo del campo excerpt

In [ ]:
count = train['excerpt'].str.split().str.len()
print("Numero di parole nel campo excerpt:\n", count)
print("Massimo numero di parole nel campo excerpt: ", max(count))

In [ ]:
train['excerpt_len'] = train['excerpt'].apply(
    lambda x: len(x)
)
train['excerpt_word_count'] = train['excerpt'].apply(
    lambda x: len(x.split(' '))
)

### Lunghezza degli estratti di testo

In [ ]:
sns.kdeplot(train['excerpt_len']).set_title("Excerpt Len distribution")
plt.show()

Numero di parole degli estratti di testo

In [ ]:
sns.kdeplot(train['excerpt_word_count']).set_title("Excerpt word Count distribution")
plt.show()

In [ ]:
sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="excerpt_len",
    kind="hex",
    height=8)
plt.suptitle("target vs excerpt_len", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

Confrontando il valore target con la lunghezza degli estratti si può notare una leggera relazione. Gli estratti più
lunghi tendono ad avere un valore target minore.

### Parole uniche per ogni estratto

In [ ]:
def uniqueWordCount(text):
    text = text.lower()
    words = text.split()
    words = [word.strip('.,!;()[]') for word in words]
    words = [word.replace("'s", '') for word in words]

    #finding unique
    unique = []
    for word in words:
        if word not in unique:
            unique.append(word)

    return len(unique)


uniqueWordCount("prova ciao Prova")
train['unique_word_count'] = train['excerpt'].apply(uniqueWordCount)
test['unique_word_count'] = test['excerpt'].apply(uniqueWordCount)

In [ ]:
sns.kdeplot(train['unique_word_count']).set_title("Unique word count distribution")
plt.show()

In [ ]:
sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="unique_word_count",
    kind="hex",
    height=8)
plt.suptitle("target vs unique words", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

## Analisi dell'errore standard
Dato che ci sono diverse valutazioni per ogni testo, lo standard_error viene in aiuto per misurare lo spread.

In [ ]:
stats.probplot(train.standard_error[train.standard_error != 0], plot=plt)
plt.show()

In [ ]:
sns.histplot(train.standard_error).set_title('standard_error distribution')
plt.show()

Ordinazione crescente dei dati in base allo standard_error

In [ ]:
display(train.sort_values(by=['standard_error']).head())

Ordinazione decrescente dei dati in base allo standard_error

In [ ]:
display(train.sort_values(by=['standard_error'], ascending=False).head())

In [ ]:
sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="standard_error",
    kind="hex",
    height=8)
plt.suptitle("target vs standard_error", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

Osservando il grafico in cui viene tracciato lo standard_error rispetto al target senza la riga base, è possibile
vedere una relazione. I testi con una facilità di lettura media tendono ad avere uno spread minore, mentre alle due
estremità abbiamo un alto spread.
Le opinioni soggettive dei valutatori variano molto quando valutano quegli estratti facili e difficili, ma danno
opinioni più ravvicinate quando gli estratti hanno una difficoltà media.

# Preprocessing

Lo scopo di questo processo è quello di pulire i dati e renderli leggibili per i modelli di machine learning che
verranno implementati.

## Data cleaning

Andiamo a pulire i dati attraverso varie funzioni. Per prima cosa rimuoviamo le "stop words".

In [ ]:
def rimuoviStopWords(testo):
    tokenized_text = wt(testo)
    sms_processed = []
    for word in tokenized_text:
        if word not in set(stopwords.words('english')):
            sms_processed.append(word)

    clean_text = " ".join(sms_processed)

    return clean_text


print("Testo prima della funzione rimuoviStopWords:\n" + train['excerpt'][1])
print(hrule(20))
print("Testo dopo la funzione:\n" + rimuoviStopWords(train['excerpt'][1]))

Rimuoviamo i segni di punteggiatura.

In [ ]:
def rimuoviPunteggiatura(testo):
    return testo.translate(str.maketrans('', '', string.punctuation))


print("Testo prima della funzione rimuoviPunteggiatura:\n" + train['excerpt'][1])
print(hrule(20))
print("Testo dopo la funzione:\n" + rimuoviPunteggiatura(train['excerpt'][1]))

Rimuoviamo eventuali link.

In [ ]:
def rimuoviLink(testo):
    clean_text = re.sub('https?://\S+|www\.\S+', '', testo)
    return clean_text


test_string = "Il link http://www.google.com/ andrebbe rimosso"
print(test_string)
print(hrule(20))
print(rimuoviLink(test_string))

Eliminiamo i numeri.

In [ ]:
def rimuoviNumeri(testo):
    clean_text = re.sub(r'\d+', '', testo)
    return clean_text


test_string = "Il numero 34 va rimosso"
print(test_string)
print(hrule(20))
print(rimuoviNumeri(test_string))

In [ ]:
def clean(testo):
    testo = testo.lower()  #Lets make it lowercase
    testo = rimuoviStopWords(testo)
    testo = rimuoviPunteggiatura(testo)
    testo = rimuoviNumeri(testo)
    testo = rimuoviLink(testo)
    return testo


train['excerpt_clean'] = train['excerpt'].apply(clean)
test['excerpt_clean'] = test['excerpt'].apply(clean)

train.head()

## Stemming

In questo processo facciamo la derivazione delle parole negli estratti. Proveremo con lo stemming, una funzione
efficiente per questo tipo di calcolo. Lo stemming produce delle parole derivate chiamate "stem". Ogni stem potrebbe
non essere una parola reale nel dizionario inglese (in questo caso i testi sono in inglese).
NLTK mette a disposizione due tipi di stemmer: Porter Stemmer e Snowball Stemmer. Snowball stemmer è leggermente
migliore rispetto al Porter Stemmer, per cui verrà utilizzato questo.

In [ ]:
stemmer = SnowballStemmer(language='english')

tokens = train['excerpt'][1].split()
clean_text = ' '

for token in tokens:
    print(token + ' --> ' + stemmer.stem(token))

In [ ]:
def stemWord(text):
    stemmer = SnowballStemmer(language='english')
    tokens = text.split()
    clean_text = ' '
    for token in tokens:
        clean_text = clean_text + " " + stemmer.stem(token)
    return clean_text


print("Testo prima della funzione stemWord: " + train['excerpt'][1])
print("Testo dopo la funzione: " + stemWord(train['excerpt'][1]))

In [ ]:
train['excerpt_clean'] = train['excerpt_clean'].apply(stemWord)
test['excerpt_clean'] = test['excerpt_clean'].apply(stemWord)

## Lemmatization

Questo processo è simile allo stemming, con la differenza che la funzione lemmatization, data una parola, dà in output
una sua derivata che è sempre una parola esistente nel dizionario inglese.

In [ ]:
def lemmatizeWord(text):
    tokens = nlp(text)
    clean_text = ' '
    for token in tokens:
        clean_text = clean_text + " " + token.lemma_

    return clean_text


print("Testo prima della funzione lemmatizeWord: " + train['excerpt'][1])
print("Testo dopo la funzione: " + lemmatizeWord(train['excerpt'][1]))

Purtroppo però, a differenza dello Stemming,
questa funzione non è efficiente e impiegherebbe molto tempo per essere eseguito su tutti i testi del dataset.

Una volta puliti gli estratti, facciamo un'ulteriore analisi per assicurarci che le correlazioni trovate continuano ad
essere valide.

In [ ]:
train['clean_excerpt_len'] = train['excerpt_clean'].apply(
    lambda x: len(x)
)

In [ ]:
train['excerpt_len'] = train['excerpt'].apply(
    lambda x: len(x)
)

sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="excerpt_len",
    kind="hex",
    height=8)
plt.suptitle("target vs excerpt_len", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="clean_excerpt_len",
    kind="hex",
    height=8)
plt.suptitle("target vs clean_excerpt_len", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

La correlazione tra la lunghezza del testo e il valore target è stata preservata. Mostriamo la matrice di correlazione
per osservare le differenze.

In [ ]:
df = pd.DataFrame(train, columns=['target', 'excerpt_len', 'clean_excerpt_len'])
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

Vediamo ora se è stata preservata la correlazione tra le il numero di parole uniche in un testo e il target relativo

In [ ]:
train['clean_unique_word_count'] = train['excerpt_clean'].apply(uniqueWordCount)

sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="unique_word_count",
    kind="hex",
    height=8)
plt.suptitle("target vs clean_excerpt_len", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

sns.jointplot(
    data=train[train.standard_error != 0],
    x="target",
    y="clean_unique_word_count",
    kind="hex",
    height=8)
plt.suptitle("target vs clean_excerpt_len", font="Serif", size=20)
plt.subplots_adjust(top=0.95)
plt.show()

df = pd.DataFrame(train, columns=['target', 'unique_word_count', 'clean_unique_word_count'])
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In questo caso la correlazione non solo è stata preservata, ma è anche aumentata dopo il processo di data cleaning

# Modelli

## Bag of words e TF-IDF

Gli algoritmi di machine learning non capiscono caratteri o parole, per questo motivo abbiamo la necessità di trasformare i nostri estratti di testo in numeri. Questo processo viene chiamato "tokenizzazione". Tra gli algoritmi di tokenizzazione più conosciuti ci sono:
* CountVectorizer
* TfidfVectorizer

Il primo semplicemente conta le occorrenze di una parola all'interno di un estratto e quindi restituisce un array di interi. Il secondo, un po' più complesso, assegna un punteggio ad ogni parola restituendo un array di float.

In [ ]:
cv = CountVectorizer(stop_words='english')
tv = TfidfVectorizer(stop_words='english')

a = "Data mining is a beautiful subject, I like it!"
b = "Data mining is the best subject"

cv_score = cv.fit_transform([a, b])
tv_score = tv.fit_transform([a, b])

def matrix_to_list(matrix):
    matrix = matrix.toarray()
    return matrix.tolist()

cv_score_list = matrix_to_list(cv_score)
tv_score_list = matrix_to_list(tv_score)

print("tfidf_a  tfidf_b  count_a count_b   word")
print("-"*41)
for i in range(6):
    print("  {:.3f}    {:.3f}        {:}       {:}   {:}".format(tv_score_list[0][i],
                                               tv_score_list[1][i],
                                               cv_score_list[0][i],
                                               cv_score_list[1][i],
                                               cv.get_feature_names()[i]))

Si può notare come il punteggio per ogni parola ottenuto con TfidfVectorizer sia particolare: le parole "best" e "data" sono contenute lo stesso numero di volte nella stringa b, eppure hanno ottenuto un punteggio diverso. Questo perché TfidfVectorizer tiene conto della frequenza del termine negli estratti di testo, infatti "best" non era contenuto nella prima stringa, mentre "data" sì.

## Linear Regression Unigram

Il primo modello è una semplice regressione lineare allenato con la rappresentazione del bag of words.

La dicitura Unigram sta a significare che nel momento in cui facciamo la tokenizzazione, dividiamo il testo parola per parola assegnando uno score ad ognuna di esse.

In [ ]:
corpus = ['This is a useful document for testing']
vectorizer = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

Sulla base di questo costruiamo il modello di regressione lineare. Per prima cosa trasformiamo i dati in dei vettori, dopo procediamo la divisione del dataset in trainset e testset.

In [ ]:
rmse = lambda y_true, y_pred: np.sqrt(mse(y_true, y_pred))
rmse_loss = lambda Estimator, X, y: rmse(y, Estimator.predict(X))

In [ ]:
x = train['excerpt_clean']
y = train['target']

print(len(x), len(y))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

Costruiamo il modello usando CountVectorizer

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1, 1)),
    LinearRegression(),
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f' RMSE: {rmse(y_test, y_pred):.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.histplot(y_test, color="green", ax=ax, label='Testset', kde=True, stat="density", linewidth=0)
sns.histplot(y_pred, color="orange", ax=ax, label='Prediction', kde=True, stat="density", linewidth=0)
ax.legend()
plt.show()

Ora ricostruiamo il modello usando questa volta TfidfVectorizer

In [ ]:
model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 1)),
    LinearRegression(),
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f' RMSE: {rmse(y_test, y_pred):.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.histplot(y_test, color="green", ax=ax, label='Testset', kde=True, stat="density", linewidth=0)
sns.histplot(y_pred, color="orange", ax=ax, label='Prediction', kde=True, stat="density", linewidth=0)
ax.legend()
plt.show()

TfidfVectorizer è migliore di CountVectorizer perché non si concentra solo sulla frequenza delle parole presenti negli estratti di testo, ma fornisce anche l'importanza delle parole. TfidfVectorizer si basa sulla logica che le parole troppo comuni e le parole troppo rare non sono entrambe statisticamente importanti per trovare uno schema. Per questo motivo, a seguire, verrà utilizzato TfidfVectorizer per i modelli di regressione lineare.

## Linear Regression Bigram

Nel Bigram la divisione del testo non avviene parola per parola come in Unigram, ma vengono prese due parole per volta.

In [ ]:
corpus = ['This is a useful document for testing']
vectorizer = TfidfVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

Costruiamo il modello

In [ ]:
model = make_pipeline(
    TfidfVectorizer(ngram_range=(2, 2)),
    LinearRegression(),
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f' RMSE: {rmse(y_test, y_pred):.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.histplot(y_test, color="green", ax=ax, label='Testset', kde=True, stat="density", linewidth=0)
sns.histplot(y_pred, color="orange", ax=ax, label='Prediction', kde=True, stat="density", linewidth=0)
ax.legend()
plt.show()

## Ridge Regression

In [ ]:
# Unigram
model = make_pipeline(
    TfidfVectorizer(binary=True, ngram_range=(1,1)),
    Ridge(fit_intercept=True, normalize=False),
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f' RMSE: {rmse(y_test, y_pred):.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.histplot(y_test, color="green", ax=ax, label='Testset', kde=True, stat="density", linewidth=0)
sns.histplot(y_pred, color="orange", ax=ax, label='Prediction', kde=True, stat="density", linewidth=0)
ax.legend()
plt.show()

## Extreme Gradient Boosting

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1, 1)),
    xgb.XGBRegressor(),
)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f' RMSE: {rmse(y_test, y_pred):.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.histplot(y_test, color="green", ax=ax, label='Testset', kde=True, stat="density", linewidth=0)
sns.histplot(y_pred, color="orange", ax=ax, label='Prediction', kde=True, stat="density", linewidth=0)
ax.legend()
plt.show()

## GloVe Embeddings

GloVe è un algoritmo di apprendimento per ottenere rappresentazioni vettoriali per le parole. L'addestramento viene eseguito su statistiche globali aggregate di co-occorrenza parola-parola (embedding) da un corpus e le rappresentazioni risultanti mostrano interessanti sottostrutture lineari dello spazio vettoriale di parole. Ma se invece di eseguire l'addestramento sugli embedding, potessi usare gli embedding già appresi, dove i ricercatori hanno già fatto il duro lavoro di trasformare le parole in vettori e quei vettori sono stati dimostrati?

I word embedding preaddestrati sono gli embedding appresi in un'attività che vengono utilizzati per risolvere un'altra attività simile. Questi embedding vengono addestrati su set di dati di grandi dimensioni, salvati e quindi utilizzati per risolvere altre attività. Gli embedding di parole pre-addestrati sono una forma di Transfer Learning.

In [ ]:
glove_embeddings = dict()
f = open('/kaggle/input/glove6b100d/glove.6B.100d.txt')
print(f)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings[word] = coefs
f.close()

Caricato il file contenenti gli embedding preaddestrati, possiamo vedere una peculiarià dei vettori di GloVe: la distanza euclidea tra due vettori di parole fornisce un metodo efficace per misurare la somiglianza linguistica o semantica delle parole corrispondenti. Ad esempio, ecco le parole più vicine alla parola "computer":

In [ ]:
computer = glove_embeddings['computer']
tolerance = 5

for w_vec in glove_embeddings:
    if distance.euclidean(computer, glove_embeddings[w_vec]) < tolerance:
        print(w_vec)

Ora possiamo creare il modello trasformando gli estratti di testo in vettori.

In [ ]:
def sent2vec(s):
    words = str(s).lower()
    words = wt(words)
    M = []
    for w in words:
        try:
            M.append(glove_embeddings[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(50)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.excerpt, train.target, random_state=42, test_size=0.1,
                                                  shuffle=True)

xtrain_glove = np.array([sent2vec(x) for x in xtrain])
xvalid_glove = np.array([sent2vec(x) for x in xvalid])

clf = Ridge(fit_intercept=True, normalize=False),
clf = xgb.XGBRegressor(max_depth=6, n_estimators=230, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(xtrain_glove, ytrain)
predictions = clf.predict(xvalid_glove)

print("RMSE: %f " % rmse(yvalid, predictions))

## Bert

Introduciamo un nuovo modello chiamato BERT, che sta per  Bidirectional Encoder Representations from Transformers. A differenza dei precedenti modelli, BERT è progettato per pre-addestrare rappresentazioni bidirezionali profonde da testo senza etichetta condizionando contemporaneamente il contesto sinistro e destro in tutti i livelli. Di conseguenza, possiamo utilizzare BERT pre-addestrato come base e con un solo livello di output aggiuntivo si possono creare modelli all'avanguardia per un'ampia gamma di attività, come risposta alle domande e inferenza linguistica, senza sostanziali modifiche specifiche dell'architettura.

BERT è concettualmente semplice ed empiricamente potente, è stato addestrato con gli obiettivi di modellazione del linguaggio mascherato (MLM) e di previsione della frase successiva (NSP). È efficiente nel prevedere token mascherati e NLU in generale, ma non è ottimale per la generazione di testo.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('../input/huggingface-bert/bert-large-uncased/vocab.txt', do_lower_case=True)

I dati sono codificati secondo i requisiti BERT. C'è una funzione molto utile chiamata encode_plus fornita nella classe Tokenizer. Può eseguire senza problemi le seguenti operazioni:

* Tokenizzare il testo
* Aggiungi token speciali per la predizione del testo - [CLS] e [SEP], il primo indica l'inizio di una prima frase e il secondo indica l'inizio delle frasi successive.
* Crea dei token ID
* Riempie le frasi con token PAD in modo che abbiano una lunghezza comune
* Crea maschere di attenzione per i token PAD di cui sopra

In [ ]:
maximum_length = 120

def bert_encode(data) :
  input_ids = []
  attention_masks = []

  for i in range(len(data.excerpt)):
      encoded = tokenizer.encode_plus(
        data.excerpt[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        return_attention_mask=True,
      )

      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids,train_attention_masks = bert_encode(train)
test_input_ids,test_attention_masks = bert_encode(test)

In [ ]:
def create_model(bert_model):
  input_ids = tf.keras.Input(shape=(maximum_length,),dtype='int32')
  attention_masks = tf.keras.Input(shape=(maximum_length,),dtype='int32')

  output = bert_model([input_ids,attention_masks])
  output = output[1]

  output = tf.keras.layers.Dense(1)(output)
  model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
  model.compile(tf.keras.optimizers.Adam(lr=6e-6), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
  return model

In [ ]:
config=BertConfig()
config.output_hidden_states=False
model = TFBertModel.from_pretrained('../input/huggingface-bert/bert-base-uncased/tf_model.h5', config=config)

In [ ]:
bert_model = create_model(model)
bert_model.summary()

In [ ]:
history = bert_model.fit([train_input_ids,train_attention_masks],train.target,validation_split=0.3, epochs=2, batch_size=10)

Distribuzione dell'errore a confronto tra il testset e la sua predizione effettuata:

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Analisi dei risultati

## Modelli TF-IDF

Abbiamo utilizzato la funzione tf-idf per misurare l'importanza dei termini rispetto alla collezione degli estratti di testo, implementandolo all'interno dei modelli
* Linear Regression
* Ridge Regression
* Extreme Gradient Boosting

In particolare nel modello di linear regression abbiamo potuto notare come Unigram fornisca un risultato migliore rispetto a Bigram. Non è comune che i modelli Bigram abbiano prestazioni peggiori dei modelli Unigram, ma ci sono situazioni in cui può accadere, in particolare, aggiungendo relazioni aggiuntive che possono portare ad overfitting. Inoltre, nell'analisi dei dati abbiamo riportato una relazione riguardante il numero di parole uniche negli estratti di testo e il target, relazione di cui tf-idf tiene conto e che probabilmente è meno accentuata utilizzando Bigram.

In seguito mostreremo una breve ricapitolazione su come si comportano i tre modelli con Unigram, Bigram e Trigram per osservarne le differenze.

In [ ]:
def training(model, X_train, y_train, X_test, y_test, model_name, ngram_range):
    t1 = time.time()

    model = make_pipeline(
        TfidfVectorizer(binary=True, ngram_range=ngram_range),
        model,
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    RMSE = rmse(y_test, y_pred)

    t2 = time.time()
    training_time = t2 - t1

    print("--- Model:", model_name, "---")
    print("RMSE: ", RMSE)
    print("Training time:", training_time)
    print("\n")

In [ ]:
lr = LinearRegression()
ridge = Ridge(fit_intercept=True, normalize=False)
xgbr = xgb.XGBRegressor()

models = [lr, ridge, xgbr]

modelnames = ["Linear Regression", "Ridge Regression", "Extreme Gradient Boosting"]

X = train["excerpt_clean"]
y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_gram_dict = {"Unigram": (1, 1), "Unigrams + Bigrams": (1, 2), "Bigrams alone": (2, 2),
               "Unigrams + Bigrams + Trigrams": (1, 3), "Trigrams alone": (3, 3)}

for n_gram in n_gram_dict.keys():
    print("\033[1m " + n_gram + " \n \033[0m")
    for i in range(0, len(models)):
        training(model=models[i], X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
                 model_name=modelnames[i], ngram_range=n_gram_dict[n_gram])
    print("*" * 40)

Il risultato migliore è ottenuto dal modello Ridge Regression Unigram con la radice dell'errore quadratico medio pari a 0.7096.

## GloVe Embeddings

La radice dell'errore quadratico medio ottenuto con GloVe è 0.6851. Per ottenere questo risultato è stato necessario aggiustare alcuni parametri in modo ottimale, ma la cosa più importante di tutte sono stati usati gli estratti di testo originali e non quelli "puliti". GloVe, infatti, è un modello basato su word embeddings e come tale riesce a scovare regolarità linguistiche[2]. Proviamo a ricostruire il modello utilizzando excerpt_clean e osserviamo le differenze.

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.excerpt_clean, train.target, random_state=42, test_size=0.1,
                                                  shuffle=True)

xtrain_glove = np.array([sent2vec(x) for x in xtrain])
xvalid_glove = np.array([sent2vec(x) for x in xvalid])

clf = Ridge(fit_intercept=True, normalize=False),
clf = xgb.XGBRegressor(max_depth=6, n_estimators=230, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(xtrain_glove, ytrain)
predictions = clf.predict(xvalid_glove)

print("RMSE: %f " % rmse(yvalid, predictions))

L'RMSE è decisamente più alto e ciò significa che attraverso il data cleaning, la sintassi degli estratti viene alterata.

### Riferimenti
[2] X. Zhu e G. de Melo, «Sentence Analogies: Linguistic Regularities in Sentence Embeddings,» in Proceedings of the 28th International Conference on Computational Linguistics, 2020, p. 3389–3400.

## BERT

Per lo stesso motivo descritto in GloVe, anche in BERT sono stati utilizzati gli estratti di testo originali e non quelli puliti. Attraverso questo modello è stato ottenuto il punteggio migliore sulla classifica pubblica della competizione con un RMSE pari a 0.590

# Submission

In [ ]:
config=BertConfig()
config.output_hidden_states=False
model = TFBertModel.from_pretrained('../input/huggingface-bert/bert-base-uncased/tf_model.h5', config=config)

In [ ]:
bert_model = create_model(model)
history = bert_model.fit([train_input_ids,train_attention_masks],train.target, epochs=2, batch_size=10)

y_pred = bert_model.predict([test_input_ids,test_attention_masks])


sample['target']= y_pred
sample.to_csv('submission.csv', index=False)
sample